# Data Preprocessing for Emotional Status Project
Author: Sicong Zhao

## 1.Load Data

In [1]:
import pickle
import numpy as np
import pandas as pd
from functools import reduce
import datetime
from tqdm import tqdm # Show pandas progress

In [2]:
# Experience Sampling data, which contains 8 metrics labels for emotion states.
# We are using these 8 metrics to generate the positive or negative of the emotion, and make the prediction.
with open('data/EXPERIENCING_SAMPLE_R00_DND.pickle', 'rb') as handle:
    exp = pickle.load(handle)

In [3]:
# Predictor: Steps, by minute, from Fitbit
# with open('data/STEPS_R00_DND.pickle', 'rb') as handle:
#     steps = pickle.load(handle)
steps = pd.read_csv('steps_with_datetime.csv')
steps['datetime'] = pd.to_datetime(steps['datetime'], errors='coerce')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Predictor: Heart Rate, by minute, from Fitbit
# with open('data/HR_R00_DND.pickle', 'rb') as handle:
#     hr = pickle.load(handle)
hr = pd.read_csv('hr_with_datetime.csv')
hr['datetime'] = pd.to_datetime(hr['datetime'], errors='coerce')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Predictor: Sleep, by day, from Fitbit
with open('data/SLEEP_R00_DND.pickle', 'rb') as handle:
    sleep = pickle.load(handle)

In [6]:
# Predicor: Survey, contains demographic, health, personality data, one time measurement for each subject.
with open('data/SURVEY_R00_DND.pickle', 'rb') as handle:
    survey = pickle.load(handle)

In [7]:
# A lot of missing data in survey
survey['AVI'].isna().sum() / survey['AVI'].shape[0]

index                          0.772152
Subject                        0.000000
Actually Feel: Enthusiastic    0.772152
Actual HAP                     0.772152
Actually Feel: Dull            0.772152
                                 ...   
LAN_ideal                      0.297468
N_ideal                        0.297468
HAN_ideal                      0.297468
HA_ideal                       0.297468
Experiment                     0.000000
Length: 139, dtype: float64

## 2.Pre-processing

### (1) Convert ID to string

In [8]:
steps['ID'] = steps['ID'].astype('str')
sleep['ID'] = sleep['ID'].astype('str')
hr['ID'] = hr['ID'].astype('str')

### (2) Add timestamp for heart rate, step data

In [9]:
# hr['datetime'] = hr['Date'].astype(str) + ' ' + hr['TIME'].astype(str)
# hr['datetime'] = pd.to_datetime(hr['datetime'], errors='coerce')
# hr.to_csv('hr_with_datetime.csv')

In [9]:
# steps['datetime'] = steps['Date'].astype(str) + ' ' + steps['TIME'].astype(str)
# steps['datetime'] = pd.to_datetime(steps['datetime'], errors='coerce')
# steps.to_csv('steps_with_datetime.csv')

### (3) Extract useful features and merge survey data

In [13]:
survey.keys()

dict_keys(['DEMOGRAPHICS', 'MEDICAL_SCREENING', 'AVI', 'BISBAS', 'BIS', 'TPQ_NS', 'NEO_SF', 'SWLS', 'FTP', 'SBQ'])

In [10]:
# Get columns for Medical Screening
med_df = survey['MEDICAL_SCREENING'][['Subject','BMI']]

In [11]:
# Get columns for SBQ
sbq_df = survey['SBQ'][['Subject', 'SBQ']]

In [12]:
# Get columns for FTP
ftp_df = survey['FTP'][['Subject', 'FTP']]

In [13]:
# Get columns for SWLS
swls_df = survey['SWLS'][['Subject', 'SWLS']]

In [14]:
# Get columns for NEO-SF
neosf_df = survey['NEO_SF'][['Subject', 'Neuroticism', 'Extraversion', 'Openness', 'Agreeableness','Conscientiousness']]

In [15]:
# Get columns for TPQ-NS
tpqns_df = survey['TPQ_NS'][['Subject','NS_total']]

In [16]:
# Get columns for BIS
bis_df = survey['BIS'][['Subject','BIS_total']]

In [17]:
# Get columns for BISBAS
bisbas_cols = ['Subject', 'BIS.5', 'BAS_D', 'BAS_FS', 'BAS_RR']
bisbas_df = survey['BISBAS'][bisbas_cols]

In [18]:
# Get columns for AVI
avi_col_actual = [x for x in survey['AVI'].columns.values if 'actual' in x]
avi_col_ideal = [x for x in survey['AVI'].columns.values if 'ideal' in x]
avi_col = np.concatenate([['Subject'], avi_col_actual,avi_col_ideal])
avi_df = survey['AVI'][avi_col]
avi_df['Subject'] = avi_df['Subject'].astype('str')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
# Get columns for Demographics
demo_df = survey['DEMOGRAPHICS'][['Subject', 'Education', 'Ethnicity', 'Sex', 'Marital_Status', 'Children', 'Household_income', 'Religion', 'Age', 'Medications']]

In [20]:
# Merge all dataframes, which contains the features for modeling
data_frames = [sbq_df, ftp_df, swls_df, neosf_df, tpqns_df, bis_df, bisbas_df, avi_df, demo_df, med_df]
for df in data_frames:
    df['Subject'] = df['Subject'].astype(str)

survey_df = reduce(lambda  left,right: pd.merge(left,right,on=['Subject'],
                                            how='left'), data_frames)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
# Convert data type
survey_df['Ethnicity'] = survey_df['Ethnicity'].astype(str, errors = 'raise')
survey_df['Sex'] = survey_df['Sex'].astype(str, errors = 'raise')
survey_df['Marital_Status'] = survey_df['Marital_Status'].astype(str, errors = 'raise')
survey_df['Household_income'] = survey_df['Household_income'].astype(str, errors = 'raise')
survey_df['Religion'] = survey_df['Religion'].astype(str, errors = 'raise')
survey_df['Medications'] = survey_df['Medications'].astype(str, errors = 'raise')

In [22]:
# Missing data detection
survey_df.isna().sum()/survey_df.shape[0]

Subject              0.000000
SBQ                  0.000000
FTP                  0.297468
SWLS                 0.297468
Neuroticism          0.297468
Extraversion         0.297468
Openness             0.772152
Agreeableness        0.772152
Conscientiousness    0.297468
NS_total             0.297468
BIS_total            0.297468
BIS.5                0.297468
BAS_D                0.297468
BAS_FS               0.297468
BAS_RR               0.297468
HAP_actual           0.297468
P_actual             0.297468
LAP_actual           0.297468
LA_actual            0.297468
LAN_actual           0.297468
N_actual             0.297468
HAN_actual           0.297468
HA_actual            0.297468
HAP_ideal            0.297468
P_ideal              0.297468
LAP_ideal            0.297468
LA_ideal             0.297468
LAN_ideal            0.297468
N_ideal              0.297468
HAN_ideal            0.297468
HA_ideal             0.297468
Education            0.000000
Ethnicity            0.000000
Sex       

In [23]:
survey_df.head()

,Subject,SBQ,FTP,SWLS,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,NS_total,...,Education,Ethnicity,Sex,Marital_Status,Children,Household_income,Religion,Age,Medications,BMI
0,1002,55.0,6.8,20.0,26.0,48.0,47.0,48.0,46.0,16.0,...,14,White,Female,Single,0.0,"$10,000-$19,999",Buddhism,32.0,None,23.725936
1,1004,45.0,5.3,25.0,28.0,48.0,46.0,49.0,49.0,18.0,...,14,White/Pacific Islander,Male,Single,0.0,"$10,000-$19,999",Questionable,28.0,None,30.221958
2,1008,57.0,5.0,24.0,28.0,47.0,28.0,50.0,48.0,11.0,...,16,White,Male,Married,2.0,"$150,000 or more",Judiasm,46.0,Aciphex,31.306605
3,1009,53.0,5.0,22.0,26.0,45.0,42.0,43.0,48.0,7.0,...,22,White,Male,Married,0.0,"$150,000 or more",Judiasm,58.0,None,28.408163
4,1014,55.0,3.7,25.0,26.0,36.0,41.0,42.0,49.0,9.0,...,18,White,Female,Married,3.0,"$120,000-$129,999",Christianity,68.0,"Lipitor, Diovan, 81mg Aspirin",31.256942


In [24]:
survey_df.dtypes

Subject               object
SBQ                  float64
FTP                  float64
SWLS                 float64
Neuroticism          float64
Extraversion         float64
Openness             float64
Agreeableness        float64
Conscientiousness    float64
NS_total             float64
BIS_total            float64
BIS.5                float64
BAS_D                float64
BAS_FS               float64
BAS_RR               float64
HAP_actual           float64
P_actual             float64
LAP_actual           float64
LA_actual            float64
LAN_actual           float64
N_actual             float64
HAN_actual           float64
HA_actual            float64
HAP_ideal            float64
P_ideal              float64
LAP_ideal            float64
LA_ideal             float64
LAN_ideal            float64
N_ideal              float64
HAN_ideal            float64
HA_ideal             float64
Education             object
Ethnicity             object
Sex                   object
Marital_Status

(6) Fix ID inconsistency

In [25]:
# Fix subject ID inconsistant issue, only run once
exp['subject'] = exp['subject'].str.replace('DND', 'DND1', regex=False)
survey_df['Subject'] = survey_df['Subject'].str.replace('DND', 'DND1', regex=False)

## 3.Feature Engineering

### (1) Time window for feature engineering

In [26]:
# Generate 30 mins prior time point
exp['start_survey_5m_ahead'] = exp['start_survey'] - datetime.timedelta(minutes=5)
exp['start_survey_10m_ahead'] = exp['start_survey'] - datetime.timedelta(minutes=10)
exp['start_survey_30m_ahead'] = exp['start_survey'] - datetime.timedelta(minutes=30)
exp['start_survey_1h_ahead'] = exp['start_survey'] - datetime.timedelta(minutes=60)
exp['start_survey_3h_ahead'] = exp['start_survey'] - datetime.timedelta(hours=3)

### (2) Step related feature

Steps related features:
* Statistics: max, min, mean, std
* Move Rate: The # of minutes with step > 0 / total minutes
* Active Rate: The # of minutes with step > 10 / total minutes
* Very Active Rate: The # of minutes with step > 20 / total minutes
* Running Rate: The # of minutes with step > 30 / total minutes

In [54]:
def step_feature_gen(df):
    if df.shape[0] == 0:
        return [None,None,None,None,None,None,None,None,None]
    else:
        max_step = df['VALUE'].max()
        min_step = df['VALUE'].min()
        mean = df['VALUE'].mean()
        var = df['VALUE'].var()
        median = df['VALUE'].median()
        move_rate = df[df['VALUE'] > 0].shape[0]/df.shape[0]
        active_rate = df[df['VALUE'] > 10].shape[0]/df.shape[0]
        very_active_rate = df[df['VALUE'] > 20].shape[0]/df.shape[0]
        running_rate = df[df['VALUE'] > 30].shape[0]/df.shape[0]
        return [max_step, min_step, mean, var, median, move_rate, active_rate, very_active_rate, running_rate]

def gen_step_features_by_id(subject_id, index, row, step_by_id):
    start_3h_time = row['start_survey_3h_ahead']
    start_1h_time = row['start_survey_1h_ahead']
    start_30m_time = row['start_survey_30m_ahead']
    start_10m_time = row['start_survey_10m_ahead']
    start_5m_time = row['start_survey_5m_ahead']
    end_time = row['start_survey']
    df_before_exp = step_by_id[step_by_id['datetime'] <= end_time]
    df_3h = df_before_exp[df_before_exp['datetime'] >= start_3h_time]
    df_1h = df_before_exp[df_before_exp['datetime'] >= start_1h_time]
    df_30m = df_before_exp[df_before_exp['datetime'] >= start_30m_time]
    df_10m = df_before_exp[df_before_exp['datetime'] >= start_10m_time]
    df_5m = df_before_exp[df_before_exp['datetime'] >= start_5m_time]
    # Generate features
    step_features_3h = step_feature_gen(df_3h)
    step_features_1h = step_feature_gen(df_1h)
    step_features_30m = step_feature_gen(df_30m)
    step_features_10m = step_feature_gen(df_10m)
    step_features_5m = step_feature_gen(df_5m)
    prev_max_step = df_before_exp['VALUE'].max()
    prev_min_step = df_before_exp['VALUE'].min()
    prev_med_step = df_before_exp['VALUE'].median()
    return [index, subject_id, prev_max_step, prev_min_step, prev_med_step] \
            + step_features_3h + step_features_1h + step_features_30m + step_features_10m + step_features_5m

Step1. Get subject id

In [28]:
steps_ids = np.unique(steps['ID'].values)
exp_ids = np.unique(exp.subject.values)
# Detect subject without experience sampling data
print(set(steps_ids) - set(exp_ids))
# Detect subject without step data
print(set(exp_ids) - set(steps_ids))

{'DND1058'}
{'DND1033', '1079', '1097'}


Step2. Calculate step features for all subject

In [55]:
# Get All Unique IDs, remove subject without step data. (we could not construct step features for them.)
exp_ids = np.unique(exp.subject.values)
exp_id_1 = np.in1d(exp_ids, 'DND1033').nonzero()[0]
exp_id_2 = np.in1d(exp_ids, '1079').nonzero()[0]
exp_id_3 = np.in1d(exp_ids, '1097').nonzero()[0]

exp_ids = np.delete(exp_ids, [exp_id_1,exp_id_2,exp_id_3])
    
# Result stores here
step_features = []
# Count for progress
count = 0
# Iterate each id, decrease redundant computation
for exp_id in exp_ids:
    # Progress output
    count += 1
    b = str(count) + '/'+ str(len(exp_ids))
    print (b, end="\r")
    # Get subject_id
    subject_id = exp_id
    # Extract exp dataframe by subject_id
    exp_by_id = exp[exp['subject'].astype(str) == subject_id]
    # Extract heart rate dataframe by subject_id
    step_by_id = steps[steps['ID'].astype(str) == subject_id]
    # Generate features for each experience sampling of each subject
    for index, row in exp_by_id.iterrows():
        feature_by_row = gen_step_features_by_id(exp_id, index, row, step_by_id)
        step_features.append(feature_by_row)

Step3. Convert results to dataframe, and drop NA

In [56]:
# steps_features = pd.DataFrame(steps_features.dropna())
steps_features_colnames = ['index', 'subject_id', 'step_max', 'step_min', 'step_median',
                'steps_max_3h','steps_min_3h','steps_mean_3h', 'steps_var_3h', 'steps_median_3h', 'move_rate_3h', 'active_rate_3h', 'very_active_rate_3h', 'running_rate_3h',
                'steps_max_1h','steps_min_1h','steps_mean_1h', 'steps_var_1h', 'steps_median_1h', 'move_rate_1h', 'active_rate_1h', 'very_active_rate_1h', 'running_rate_1h',
                'steps_max_30m','steps_min_30m','steps_mean_30m', 'steps_var_30m', 'steps_median_30m', 'move_rate_30m', 'active_rate_30m', 'very_active_rate_30m', 'running_rate_30m',
                'steps_max_10m','steps_min_10m','steps_mean_10m', 'steps_var_10m', 'steps_median_10m', 'move_rate_10m', 'active_rate_10m', 'very_active_rate_10m', 'running_rate_10m',
                'steps_max_5m','steps_min_5m','steps_mean_5m', 'steps_var_5m', 'steps_median_5m', 'move_rate_5m', 'active_rate_5m', 'very_active_rate_5m', 'running_rate_5m']
step_features_df = pd.DataFrame(step_features)
step_features_df.columns = steps_features_colnames
# Drop NA
step_features_cl = step_features_df.dropna()
step_features_cl.to_csv('step_features_clean.csv')

### (3) Heart Rate related features

Heart Rate related features:
* Statistics: max, min, mean, std
* Resting Rate: The # of minutes with HR < 30 percentile heart rate > 0 / total minutes
* Moderate Rate: The # of minutes with HR > 50 percentile heart rate > 0 / total minutes
* Very Active Rate: The # of minutes with HR > 80 percentile heart rate > 0 / total minutes
* SDNN: Standard deviation of heartbeat intervals
* pHR2: Percentage of the difference between ajacent HR greater than 2
* rMSSD: Root of mean squared HR change
* Highest HR
* Lowest HR
* l_h: Lowest HR / Highest HR
* CR: Highest HR / Highest HR so far

In [50]:
def hr_feature_gen(df, rest_thred, moderate_thred, active_thred, hr_max):
    if df.shape[0] == 0:
        return [None, None, None, None, None, None, None, None, None, None, None, None, None, None]
    else:
        mean = df['VALUE'].mean()
        var = df['VALUE'].var()
        std = df['VALUE'].std()
        median = df['VALUE'].median()
        rest_rate = df[df['VALUE'] <= rest_thred].shape[0]/df.shape[0]
        moderate_rate = df[df['VALUE'] > moderate_thred].shape[0]/df.shape[0]
        very_active_rate = df[df['VALUE'] > active_thred].shape[0]/df.shape[0]
        
        hr_interval_all = np.array([])
        for index, row in df.iterrows():
            hr_by_min = row['VALUE']
            hr_interval_min = np.full((1, hr_by_min), 1/hr_by_min)[0]
            hr_interval_all = np.concatenate([hr_interval_min, hr_interval_all])
        # SDNN - Standard deviation of heartbeat intervals
        SDNN = hr_interval_all.std()
        # pHR2 - percentage of the difference between ajacent HR greater than 2
        hr_seq = df['VALUE'].values
        hr_diff = abs(hr_seq[:-1] - hr_seq[1:])
        # rMSSD
        if len(hr_diff) == 0:
            pHR2 = None
            rMSSD = None
        else:
            pHR2 = sum(hr_diff > 2)/len(hr_diff)
            rMSSD = np.sqrt(sum(hr_diff**2)/len(hr_diff))
            # Lowest heart rate
        low_hr = df['VALUE'].min()
        # Highest heart rate
        high_hr = df['VALUE'].max()
        # LR/HR
        if high_hr == 0:
            l_h = None
        else:
            l_h = low_hr/high_hr
        # CR - high_hr/hr_max_of_all_time
        if hr_max == 0:
            CR = None
        else:
            CR = high_hr/hr_max
        
        return [SDNN, pHR2, rMSSD, low_hr, high_hr, l_h, CR, mean, var, std, median, rest_rate, moderate_rate, very_active_rate]

def gen_hr_features_by_id(subject_id, index, row, hr_by_id):
    start_3h_time = row['start_survey_3h_ahead']
    start_1h_time = row['start_survey_1h_ahead']
    start_30m_time = row['start_survey_30m_ahead']
    start_10m_time = row['start_survey_10m_ahead']
    start_5m_time = row['start_survey_5m_ahead']
    end_time = row['start_survey']
    
    df_before_exp = hr_by_id[hr_by_id['datetime'] <= end_time]
    hr_percentile = df_before_exp['VALUE'].quantile([0, .3, .5, .8, 1]).values
    rest_thred = hr_percentile[1]
    moderate_thred = hr_percentile[2]
    active_thred = hr_percentile[3]

    df_3h = df_before_exp[df_before_exp['datetime'] >= start_3h_time]
    df_1h = df_before_exp[df_before_exp['datetime'] >= start_1h_time]
    df_30m = df_before_exp[df_before_exp['datetime'] >= start_30m_time]
    df_10m = df_before_exp[df_before_exp['datetime'] >= start_10m_time]
    df_5m = df_before_exp[df_before_exp['datetime'] >= start_5m_time]
    # All time features
    hr_max = df_before_exp['VALUE'].max()
    hr_min = df_before_exp['VALUE'].min()
    hr_med = df_before_exp['VALUE'].median()
    # Generate features
    hr_features_3h = hr_feature_gen(df_3h, rest_thred, moderate_thred, active_thred, hr_max)
    hr_features_1h = hr_feature_gen(df_1h, rest_thred, moderate_thred, active_thred, hr_max)
    hr_features_30m = hr_feature_gen(df_30m, rest_thred, moderate_thred, active_thred, hr_max)
    hr_features_10m = hr_feature_gen(df_10m, rest_thred, moderate_thred, active_thred, hr_max)
    hr_features_5m = hr_feature_gen(df_5m, rest_thred, moderate_thred, active_thred, hr_max)
    
    return [index, subject_id, hr_max, hr_min, hr_med] + [hr_percentile[0],hr_percentile[1],hr_percentile[2],hr_percentile[3],hr_percentile[4]]\
            + hr_features_3h + hr_features_1h + hr_features_30m + hr_features_10m + hr_features_5m

In [47]:
# Detect ids in hr but not in exp
hr_ids = np.unique(hr['ID'].values)
exp_ids = np.unique(exp.subject.values)
print(set(hr_ids) - set(exp_ids))
# Detect ids in exp but not in hr
print(set(exp_ids) - set(hr_ids))

{'DND1058'}
{'DND1033'}


Step1. Get Subject ID

In [51]:
# Get All Unique IDs
exp_ids = np.unique(exp.subject.values)
exp_id_1 = np.in1d(exp_ids, 'DND1033').nonzero()[0]
exp_ids = np.delete(exp_ids, exp_id_1)

Step2. Calculate features by subject.

In [52]:
# Count for progress
count = 0
# Result stores here
hr_features = []
# Iterate each id, decrease redundant computation
for exp_id in exp_ids:
    # Progress output
    count += 1
    b = str(count) + '/'+ str(len(exp_ids))
    print (b, end="\r")
    # Get subject_id
    subject_id = exp_id
    # Extract exp dataframe by subject_id
    exp_by_id = exp[exp['subject'].astype(str) == subject_id]
    # Extract heart rate dataframe by subject_id
    hr_by_id = hr[(hr['ID'].astype(str) == subject_id)]
    # Generate features for each experience sampling of each subject
    for index, row in exp_by_id.iterrows():
        feature_by_row = gen_hr_features_by_id(exp_id, index, row, hr_by_id)
        hr_features.append(feature_by_row)

Step3. Convert results to dataframe, and drop NA

In [53]:
hr_features_colnames = ['index', 'subject_id', 'hr_max', 'hr_min', 'hr_med', 'hr_0','hr_0.3','hr_0.5','hr_0.8','hr_1',
                         'SDNN_3h', 'pHR2_3h', 'rMSSD_3h', 'low_hr_3h', 'high_hr_3h', 'l_h_3h', 'CR_3h',
                         'hr_mean_3h', 'hr_var_3h', 'hr_std_3h', 'hr_median_3h', 'hr_rest_rate_3h', 'hr_moderate_rate_3h', 'hr_very_active_rate_3h',
                         'SDNN_1h', 'pHR2_1h', 'rMSSD_1h', 'low_hr_1h', 'high_hr_1h', 'l_h_1h', 'CR_1h',
                         'hr_mean_1h', 'hr_var_1h', 'hr_std_1h', 'hr_median_1h', 'hr_rest_rate_1h', 'hr_moderate_rate_1h', 'hr_very_active_rate_1h',
                         'SDNN_30m', 'pHR2_30m', 'rMSSD_30m', 'low_hr_30m', 'high_hr_30m', 'l_h_30m', 'CR_30m',
                         'hr_mean_30m', 'hr_var_30m', 'hr_std_30m', 'hr_median_30m', 'hr_rest_rate_30m', 'hr_moderate_rate_30m', 'hr_very_active_rate_30m',
                         'SDNN_10m', 'pHR2_10m', 'rMSSD_10m', 'low_hr_10m', 'high_hr_10m', 'l_h_10m', 'CR_10m',
                         'hr_mean_10m', 'hr_var_10m', 'hr_std_10m', 'hr_median_10m', 'hr_rest_rate_10m', 'hr_moderate_rate_10m', 'hr_very_active_rate_10m',
                         'SDNN_5m', 'pHR2_5m', 'rMSSD_5m', 'low_hr_5m', 'high_hr_5m', 'l_h_5m', 'CR_5m',
                         'hr_mean_5m', 'hr_var_5m', 'hr_std_5m', 'hr_median_5m', 'hr_rest_rate_5m', 'hr_moderate_rate_5m', 'hr_very_active_rate_5m']
hr_features_df = pd.DataFrame(hr_features)
hr_features_df.columns = hr_features_colnames
# Drop NA
hr_features_cl = hr_features_df.dropna()
# Export
hr_features_cl.to_csv('hr_features_clean.csv')

In [238]:
pd.DataFrame(exp_steps_hr.isna().sum() / exp_steps_hr.shape[0])

,0
index_x,0.218618
subject_id_x,0.218618
hr_mean_3h,0.218618
hr_var_3h,0.218618
hr_std_3h,0.218618
...,...
survey_date,0.002595
start_survey_10m_ahead,0.002595
start_survey_30m_ahead,0.002595
start_survey_1h_ahead,0.002595


## 4. Merge Results

In [64]:
# Reset index for merging
step_features_cl = step_features_cl.set_index('index')
hr_features_cl = hr_features_cl.set_index('index')
# Convert id column data type for merging
exp_steps_hr['subject'] = exp_steps_hr['subject'].astype(str)
sleep['ID'] = sleep['ID'].astype(str)
# Create 'survey_date' in exp for merging with sleep
exp['start_survey'] = pd.to_datetime(exp['start_survey'])
exp['survey_date'] = pd.to_datetime(exp['start_survey'].dt.date)
# Add 'survey_date' to sleep data
sleep['Date'] = pd.to_datetime(sleep['Date'])
sleep['Date'] = pd.to_datetime(sleep['Date'].dt.date)

In [65]:
# Merge design matrix progressively
exp_steps = pd.merge(exp, step_features_cl, how='left', left_index=True, right_index=True)
exp_steps_hr = pd.merge(exp_steps, hr_features_cl, how='left', left_index=True, right_index=True)
exp_steps_hr_sleep = pd.merge(exp_steps_hr, sleep, how='left', left_on=['subject','survey_date'], right_on=['ID','Date'])

In [278]:
exp_steps_hr_sleep.to_csv('exp_steps_hr_sleep.csv')

In [66]:
exp_steps_hr_sleep_survey = pd.merge(exp_steps_hr_sleep, survey_df, how='left', left_on=['subject'], right_on=['Subject'])

In [67]:
# Add survey hour as a feature
exp_steps_hr_sleep_survey['survey_hour'] = exp_steps_hr_sleep_survey.start_survey.dt.hour
# Export data
exp_steps_hr_sleep_survey.to_csv('exp_steps_hr_sleep_survey.csv')

In [68]:
# Missing data detection
missing_df = exp_steps_hr_sleep_survey.isna().sum()/exp_steps_hr_sleep_survey.shape[0]
# Export missing data
pd.DataFrame(missing_df).to_csv('missingness.csv')